# Aufbau der Notebooks [Philipp]

- Multivalue bei Notebooks angucken
- Wie installiere ich den "scheiß"
- Widgets: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html


In [1]:
from enum import Enum
import ipywidgets as widgets
from pandas import read_csv
import pandas as pd
import asammdf
from IPython.display import display
import os
import numpy as np
import time
import pathlib as pl

ldf is not supported
xls is not supported
xlsx is not supported
yaml is not supported


## 1. Aufgabe und Daten erklären/beschreiben [Philipp]


In [2]:
# Globals

PATH_RAW_DATA = "./data/raw/"
PATH_FEATURE_DATA = "./data/"
PATH_EXPLORATION_DATA = "./exploration/"
PATH_MODEL = "./models/"
DATA_SOURCE_KIDAQ = ["TEST_NAME", "TEST_TYPE", "RPM", "FLOW_RATE", "P1", "P2"]
RAW_DATA_TYPE = ["KIDAQ", "VIB"]

DATA_SOURCE_VID = [
    "TEST_NAME",
    "TEST_TYPE",
    "RPM",
    "FLOW_RATE",
    "S1",
    "S2",
    "S3",
    "S4",
    "S5",
    "S6",
    "S7",
    "S8",
]
FEATURE = [
    "STD",
    "RANGE",
    "IQR",
    "MEAN_MEDIAN",
    "FFT",
]
OPERATING_POINT_FREQ = [725, 1450, 2175, 2900]
OPERATING_POINT_FLOW_RATE = [0, 25, 50, 75, 100]

LEARNER = Enum("LEARNER", ["DNN", "DT"])

DEFAULT_LEARNER = LEARNER.DNN
DEFAULT_RAW_DATA_TYPE = RAW_DATA_TYPE[0]
DEFAULT_RAW_DATA = DATA_SOURCE_KIDAQ

DEFAULT_CLASS_LABEL = "TEST_TYPE"

tags = widgets.TagsInput(
    value=DEFAULT_RAW_DATA, allowed_tags=DEFAULT_RAW_DATA, allow_duplicates=False
)
display(tags)

TagsInput(value=['TEST_NAME', 'TEST_TYPE', 'RPM', 'FLOW_RATE', 'P1', 'P2'], allow_duplicates=False, allowed_ta…

## 3. Preprocessing [Valerij]

Aufgeteilt nach KIDAQ und VIB (separat um es einfach zu halten)

- Auswahl der Fenstergröße in Millisekunden
- Auswahl der Abtastrate
- Multi-Selektion der Aggregation (avg, mean, std, ...)
- Frequenzanalyse
- Fourier-Transformation
- Fenstergröße nach Frequenzbereichen

### 3.1 Vorbereitung der Tainings- und Testdaten

Multi-Selektion für:

- Features
- Betriebspunkte (RPM/FLOWRATE)
- Klassifikationsarten (Szenario / Testdurchlauf)


# Vorauswahl der Feature Einstellungen und Auswahl der Daten

In [3]:
WINDOW_SIZE_MS = ["100", "200", "300", "400", "500", "600", "700", "800","900","1000"]
AGGREGATIONS = ["std", "range", "iqr", "median"]

win_sizes = widgets.Dropdown(
    placeholder= "Choose the window size in ms",
    options = WINDOW_SIZE_MS,
    description = "Window size in ms: ",
    ensure_option=True,
    disabled = False
)
display(win_sizes)

raw_data_type = widgets.Dropdown(
    placeholder= "Choose the raw data type",
    options = RAW_DATA_TYPE,
    description = "Raw data type: ",
    ensure_option=True,
    disabled = False
)
display(raw_data_type)

# list all available directories in data/raw with max depth of 1
raw_data_folders = [f.name for f in os.scandir(PATH_RAW_DATA) if f.is_dir()]
raw_data_folder = widgets.Dropdown(
    placeholder= "Choose the raw data folder",
    options = raw_data_folders,
    description = "Raw data folder: ",
    ensure_option=True,
    disabled = False
)
display(raw_data_folder)





Dropdown(description='Window size in ms: ', options=('100', '200', '300', '400', '500', '600', '700', '800', '…

Dropdown(description='Raw data type: ', options=('KIDAQ', 'VIB'), value='KIDAQ')

Dropdown(description='Raw data folder: ', options=('Setup-I',), value='Setup-I')

## Definition der Funktionen zum Preprocessing der verschiedenen Datentypen

In [4]:

from tqdm.notebook import tqdm
COLUMNS_KiDAQ = [
    "p1",
    "p2",
    "a2",
    "T2",
    "T1"
]


CWD = pl.Path.cwd()

PATH_RAW_DATA = CWD / "data" / "raw"
PATH_TO_SETUP = PATH_RAW_DATA / raw_data_folder.value 


if raw_data_type.value == "KIDAQ":
    raw_files = [file for file in PATH_TO_SETUP.glob("**/*.mf4") if "KiDAQ" in file.parts]    
elif raw_data_type.value == "VIB":
    raw_files = [file for file in PATH_TO_SETUP.glob("**/*.csv") if "Rohdaten CSV" in file.parts]
def process_vib(file):
    # error_type is the first folder name after the setup folder
    error_type_with_number = file.parts[8]
    error_type = error_type_with_number.split(" ")[0]
    rpm = file.parts[10].split("r")[0]
    rpm_percent = file.parts[11].split("%")[0]
    sensor = file.parts[-1].split(" ")[0]
    version = file.parts[12] if file.parts[12] != None else '0'

    df = read_csv(file, skiprows=2, encoding="ISO-8859-1", sep=";")

    df["Timestamp [ns]"] = pd.to_datetime(df["Timestamp [ns]"], unit="ns")
    df = df.set_index("Timestamp [ns]")
    resampled = df.resample("1s")

    df_mean = resampled.mean()
    df_mean = df_mean.rename(columns={"Value": f"{sensor}_mean"})
    df_range = resampled.max() - resampled.min()
    df_range = df_range.rename(columns={"Value": f"{sensor}_range"})
    df_std = resampled.std()
    df_std = df_std.rename(columns={"Value": f"{sensor}_std"})
    df_iqr = resampled.quantile(0.75) - resampled.quantile(0.25)
    df_iqr = df_iqr.rename(columns={"Value": f"{sensor}_iqr"})
    df_mean_median = resampled.mean() - resampled.median()
    df_mean_median = df_mean_median.rename(columns={"Value": f"{sensor}_mean_median"})

    df = pd.concat([df_mean, df_range, df_std, df_iqr, df_mean_median], axis=1)
    df["Fehlertyp"] = error_type_with_number
    df["Fehlertyp_allgemein"] = error_type
    df["rpm"] = rpm
    df["rpm%"] = rpm_percent
    df["version"] = version
    df = df.reset_index()
    df = df.drop(columns=["Timestamp [ns]"])
    df["ID"] = df.index
    df = df.melt(
        id_vars=["ID", "Fehlertyp", "Fehlertyp_allgemein", "rpm", "rpm%", "version"],
        var_name="Aggregation",
        value_name="Value",
    )

    return df 

    


def process_kidaq(file):
    
    error_type_with_number = file.parts[8]
    error_type = error_type_with_number.split(" ")[0]
    rpm = file.parts[10].split("r")[0]
    rpm_percent = file.parts[11].split("m")[1].split("%")[0]
    
    mdf = asammdf.MDF(file)
    df = mdf.to_dataframe()
    df = df.reset_index()

    df = df[COLUMNS_KiDAQ]

    df_features = None

    windows = df.groupby(df.index // (int(win_sizes.value) * 20))

    for _, window in windows:

        features = {
            "Fehlertyp": error_type_with_number, 
            "Fehlertyp_allgemein": error_type,
            "rpm": rpm, 
            "rpm%": rpm_percent
            }

        features["p1_std"] = window["p1"].std()
        features["p2_std"] = window["p2"].std()
        features["a2_std"] = window["a2"].std()
        features["T2_std"] = window["T2"].std()
        features["T1_std"] = window["T1"].std()
        
        features["p1_range"] =  window["p1"].max() - window["p1"].min()
        features["p2_range"] =  window["p2"].max() - window["p2"].min()
        features["a2_range"] =  window["a2"].max() - window["a2"].min()
        features["T2_range"] =  window["T2"].max() - window["T2"].min()
        features["T1_range"] =  window["T1"].max() - window["T1"].min()
        
        features["p1_iqr"] = window["p1"].quantile(0.75) - window["p1"].quantile(0.25)
        features["p2_iqr"] = window["p2"].quantile(0.75) - window["p2"].quantile(0.25)
        features["a2_iqr"] = window["a2"].quantile(0.75) - window["a2"].quantile(0.25)
        features["T2_iqr"] = window["T2"].quantile(0.75) - window["T2"].quantile(0.25)
        features["T1_iqr"] = window["T1"].quantile(0.75) - window["T1"].quantile(0.25)
        
        features["p1_mean_median"] = window["p1"].mean() - window["p1"].median()
        features["p2_mean_median"] = window["p2"].mean() - window["p2"].median()
        features["a2_mean_median"] = window["a2"].mean() - window["a2"].median()
        features["T2_mean_median"] = window["T2"].mean() - window["T2"].median()
        features["T1_mean_median"] = window["T1"].mean() - window["T1"].median()

        if df_features is None:
            df_features = pd.DataFrame(features, index=[0])    
        else:
            df_features = pd.concat([df_features, pd.DataFrame(features, index=[0])])

    return df_features


# Funktionem zum Preprocessing der Daten in den Frequenbereich und Generierung der Features

In [7]:
def process_freq_kidaq(file):

    error_type_with_number = file.parts[8]
    error_type = error_type_with_number.split(" ")[0]
    rpm = file.parts[10].split("r")[0]
    rpm_percent = file.parts[11].split("m")[1].split("%")[0]
    
    mdf = asammdf.MDF(file)
    df = mdf.to_dataframe()
    df.index = pd.to_timedelta(df.index, unit="s")

    resample_time = df.resample("1s")

    r = resample_time.aggregate(lambda sample: np.fft.fft)

    
    df_fft = pd.DataFrame(np.fft.fft(df))

    df_fft.columns = df.columns
    df_fft.index = df.index
    df_fft.index = pd.to_timedelta(df_fft.index, unit="s")
    df_fft = df_fft.apply(np.abs)
    resampled = df_fft.resample("1s")

    df_mean = resampled.mean().to_numpy()
    df_range = resampled.max() - resampled.min()
    df_range = df_range.to_numpy()
    df_std = resampled.std().to_numpy()
    df_iqr = resampled.quantile(0.75) - resampled.quantile(0.25)
    df_iqr = df_iqr.to_numpy()
    df_mean_median = resampled.mean() - resampled.median()
    df_mean_median = df_mean_median.to_numpy()

    data = np.concatenate((np.repeat([[error_type_with_number, error_type, rpm, rpm_percent]], df_mean.shape[0], axis=0), df_mean, df_range, df_std, df_iqr, df_mean_median), axis=1)

    with open("fft_test.csv", "a") as f:
        for feature in data:
            f.write(";".join(feature) + "\n")    

    



for file in tqdm(raw_files):
    df = process_freq_kidaq(file) 

  0%|          | 0/20 [00:00<?, ?it/s]

timestamps
0 days 00:00:00    [[(42.29611140023917+0j), (-15.242247319128994...
0 days 00:00:01    [[(42.33119121013442+0j), (-15.248076272738823...
0 days 00:00:02    [[(42.856845155358315+0j), (-15.2365759376517+...
0 days 00:00:03    [[(42.365684810094535+0j), (-15.27886313085978...
0 days 00:00:04    [[(41.86934561282396+0j), (-15.22598530448626+...
                                         ...                        
0 days 00:11:55    [[(44.34265058208257+0j), (-15.830617847444534...
0 days 00:11:56    [[(44.22988194972277+0j), (-15.810493617696508...
0 days 00:11:57    [[(43.853430634364486+0j), (-15.87418575277656...
0 days 00:11:58    [[(44.63987521547824+0j), (-15.824939200804184...
0 days 00:11:59    [[(43.937125166878104+0j), (-15.84414651064455...
Freq: S, Length: 720, dtype: object
timestamps
0 days 00:00:00    [[(39.49875782150775+0j), (-14.079091977710231...
0 days 00:00:01    [[(39.515013008378446+0j), (-14.10634013549823...
0 days 00:00:02    [[(39.53751224279404+0j), 

In [14]:

FREQ_WINDOWS = 16

def process_freq_kidaq(file):

    error_type_with_number = file.parts[8]
    error_type = error_type_with_number.split(" ")[0]
    rpm = file.parts[10].split("r")[0]
    rpm_percent = file.parts[11].split("m")[1].split("%")[0]
    
    mdf = asammdf.MDF(file)
    df = mdf.to_dataframe()
    df = df.reset_index()

    df = df[COLUMNS_KiDAQ]

    df_features = None

    windows = df.groupby(df.index // (int(win_sizes.value) * 20))

    for _, window in windows:

        for col in window.columns:
            X = np.fft.fft(window[col])
            amps = np.abs(X)

            xf = np.fft.fftfreq(len(window[col]), 1 / 20000)

            idxMax = np.argmax(amps)

            if df_features is None:
                df_features = np.concatenate(([xf[idxMax]], [amps[idxMax]]), axis=0)
            else:
                df_features = np.concatenate((df_features, [xf[idxMax]], [amps[idxMax]]), axis=0)

            freqs_window_size = len(amps) / FREQ_WINDOWS

            for x in range(FREQ_WINDOWS):
                freq_window = amps[int(x*freqs_window_size):int((x+1)*freqs_window_size)]
                df_features = np.concatenate((df_features, [np.max(freq_window, axis=0)], [np.average(freq_window, axis=0)], [np.mean(freq_window, axis=0)] ), axis=0)

    df_features = np.concatenate(([error_type_with_number], [error_type], [rpm], [rpm_percent], df_features), axis=0) 

    with open(f"data/freq_{raw_data_type}_features.csv", "w") as f:
        f.write(";".join(df_features))

    # for _, window in windows:

    #         for col in window.columns:
    #             X = np.fft.fft(window[col])
    #             amps = np.abs(X)

    #             xf = np.fft.fftfreq(len(window[col]), 1 / 20000)

    #             idxMax = np.argmax(amps)

    #             # create a dataframe with two columns amps and xf
    #             df_fft = pd.DataFrame({'amps': amps, 'xf': xf})

    #             freqs_window_size = len(amps) / FREQ_WINDOWS

    #             for x in range(FREQ_WINDOWS):
    #                 freq_window = amps[int(x*freqs_window_size):int((x+1)*freqs_window_size)]
    #                 window_features = {
    #                     "Fehlertyp": error_type_with_number,
    #                     "Fehlertyp_allgemein": error_type,
    #                     "rpm": rpm,
    #                     "rpm%": rpm_percent,
    #                     "xf_max": xf[idxMax],
    #                     "amps_max": amps[idxMax],
    #                     "amps_max_window": np.max(freq_window, axis=0),
    #                     "amps_avg_window": np.average(freq_window, axis=0),
    #                     "amps_mean_window": np.mean(freq_window, axis=0)
    #                 }

    #                 if df_features is None:
    #                     df_features = pd.DataFrame(window_features, index=[0])
    #                 else:
    #                     df_features = pd.concat([df_features, pd.DataFrame(window_features, index=[0])])


    # if not pl.Path(f"freq_{raw_data_type}_1_features.csv").exists():
    #     df_features.to_csv(f"data/freq_{raw_data_type}_features.csv", header=True, index=False, sep=";")
    # else:
         
    #     df_features.to_csv(f"freq_{raw_data_type}_1_features.csv", mode='a', header=False, index=False, sep=";")


        
for file in tqdm(raw_files):
    process_freq_kidaq(file)



  0%|          | 0/20 [00:00<?, ?it/s]

## Durch ausführen der folgenden Zelle, werden die Daten mit den ausgewählten Parametern vorbereitet und abgespeichert

In [10]:
from tqdm.notebook import tqdm

RESULT_FILE = CWD / "data" / f"{raw_data_type.value}_features.csv"

if raw_data_type.value == "VIB":
    result = None
    for file in tqdm(raw_files):
        if "VIB" in file.name:
            if result is None:
                result = process_vib(file)
            else:
                result = pd.concat([result, process_vib(file)])


    #create an ascending index to be able to pivot the dataframe
    result = result.pivot(index=["ID","Fehlertyp", "Fehlertyp_allgemein", "rpm", "rpm%", "version"], columns="Aggregation", values="Value")
    result = result.drop(columns=["ID"])
    # save the result to a csv file
    result.to_csv(RESULT_FILE, sep=";")


elif raw_data_type.value == "KIDAQ":
    for file in tqdm(raw_files):
        result = process_kidaq(file)


    if result is not None:
        if not pl.Path(RESULT_FILE).exists():
            result.to_csv(RESULT_FILE, index=False, sep=";")
        else:
            result.to_csv(RESULT_FILE, mode="a", header=False, index=False, sep=";")

  0%|          | 0/540 [00:00<?, ?it/s]

# Die folgenden Zellen dienen zum Filtern des ausgewählten Datensatzes nach den gewünschten Parametern

Hier lassen sich ausschnitte aus dem kompletten Feature-Set auswählen um zum Beispiel Training und Testdaten zu generieren

In [16]:
feature_types = ["KIDAQ", "VIB"]

f_type = widgets.Dropdown(
    options=feature_types,
    value=feature_types[0],
    description="Feature Type:",
    disabled=False,
)
display(f_type)





Dropdown(description='Feature Type:', options=('KIDAQ', 'VIB'), value='KIDAQ')

In [17]:

df_features = pd.read_csv(CWD / "data" / f"{f_type.value}_features.csv", sep=",")

error_types = df_features["Fehlertyp"].unique().tolist()
rpm_values = [725, 1450, 2175, 2900]
rpm_percent_values = [0, 25, 50, 75, 100]

columns = df_features.columns.tolist()


error_type = widgets.SelectMultiple(
    options=error_types,
    value=error_types,
    description="Error Type:",
    disabled=False,
)

rpm = widgets.SelectMultiple(
    options=rpm_values,
    value=rpm_values,
    description="RPM:",
    disabled=False,
)

rpm_percent = widgets.SelectMultiple(
    options=rpm_percent_values,
    value=rpm_percent_values,
    description="RPM%:",
    disabled=False,
)

display(error_type)
display(rpm)
display(rpm_percent)



SelectMultiple(description='Error Type:', index=(0,), options=('Fehlausrichtung 1',), value=('Fehlausrichtung …

SelectMultiple(description='RPM:', index=(0, 1, 2, 3), options=(725, 1450, 2175, 2900), value=(725, 1450, 2175…

SelectMultiple(description='RPM%:', index=(0, 1, 2, 3, 4), options=(0, 25, 50, 75, 100), value=(0, 25, 50, 75,…

In [20]:

# filter the dataframe based on the selected rpm and rpm%
df_features = df_features[df_features["Fehlertyp"].isin(error_type.value)]
df_features = df_features[df_features["rpm"].isin(rpm.value)]
df_features = df_features[df_features["rpm%"].isin(rpm_percent.value)]

# save the filtered dataframe to a csv file and let the user type in a name for the file
file_name = widgets.Text(
    value="",
    placeholder="Type something",
    description="File Name:",
    disabled=False,
)
display(file_name)


Text(value='', description='File Name:', placeholder='Type something')

In [21]:
df_features.to_csv(CWD / "data" / f"{file_name.value}.csv", index=False, sep=";")

## 4. Deskriptive/Explorative Datenanalyse [Philipp]

- Plots
- Beschreibung der Plots


## 5. Machine Learning [Kevin]

Multi-Selektion für:

- Auswahl der Featuredateien (Train/Testdaten)
- Auswahl des Learners
- Konfiguration der Hyperparameter
- Live-Validation des Models mit vorausgewählten Testdaten (Random-Search, ...)


In [ ]:
# SELECT TRAINING AND TEST DATA
featureDataDir = list(filter(lambda x: os.path.isfile(os.path.join(PATH_FEATURE_DATA, x)) and DEFAULT_RAW_DATA_TYPE in x.upper(), os.listdir(PATH_FEATURE_DATA)))

modelDropdown = widgets.Dropdown(description="training features")
modelDropdown.options = featureDataDir
selectedModelFile = None
def onTrainigFileChange(change):
    global selectedModelFile
    selectedModelFile = change['new']
modelDropdown.observe(onTrainigFileChange, names='value')
display(modelDropdown)

testFileDropdown = widgets.Dropdown(description="test features")
testFileDropdown.options = featureDataDir
selectedTestFile = None
def onTestFileChange(change):
    global selectedTestFile
    selectedTestFile = change['new']
testFileDropdown.observe(onTestFileChange, names='value')
display(testFileDropdown)


In [ ]:
# SET ML CONFIG
FEATURE_TRAIN_FILE = PATH_FEATURE_DATA + selectedModelFile
FEATURE_TEST_FILE = PATH_FEATURE_DATA + selectedTestFile

DEFAULT_OPERATING_POINTS = OPERATING_POINT_FREQ + OPERATING_POINT_FLOW_RATE


# DNN
DNN_EXPLORATION_TARGET_VAL_ACCURACY = 0.9
DNN_EXPLORATION_MAX_ITER = 1
DNN_EXPLORATION_HIDDEN_LAYERS_MIN = 2
DNN_EXPLORATION_HIDDEN_LAYERS_MAX = 4
DNN_EXPLORATION_NEURONS_MIN = 8
DNN_EXPLORATION_NEURONS_MAX = 64


DNN_EARLY_STOPPING_PATIENCE = 50
DNN_VERBOSE = 0
DNN_EPOCHS = 100
DNN_BATCH_SIZE = 32
DNN_BATCH_NORMALIZATION = True


# DT
DT_MAX_DEPTH = 6
DT_NUM_ESTIMATORS = 100

In [ ]:
# LOAD DATA
train_data = read_csv(FEATURE_TRAIN_FILE, header=None, delimiter=",", decimal='.', low_memory=False).values
test_data = read_csv(FEATURE_TEST_FILE, header=None, delimiter=",", decimal='.', low_memory=False).values
print(train_data[1:, :3])

train_x, train_y = train_data[1:, 3:].astype('float32'), train_data[1:, :1]
test_x, test_y = test_data[1:, 3:].astype('float32'), test_data[1:, :1]

In [ ]:
# Train Decision Tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


labelEncoder = LabelEncoder()
labelEncoder = labelEncoder.fit(np.ravel(train_y))
label_encoded_train_y = labelEncoder.transform(np.ravel(train_y))
label_encoded_test_y = labelEncoder.transform(np.ravel(test_y))


xgb = XGBClassifier(
    tree_method="hist",
    enable_categorical=True,
    max_depth=DT_MAX_DEPTH,
    n_estimators=DT_NUM_ESTIMATORS,
)
# fit model
xgb.fit(train_x, label_encoded_train_y, eval_set=[(test_x, label_encoded_test_y)])


preds = xgb.predict(test_x)
accuracy = accuracy_score(label_encoded_test_y, preds)

XGBClassifier.save_model(xgb, PATH_MODEL+"dt."+str(round(time.time())))

In [ ]:
# Train Neural Network
import json
import random
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

onehotencoder = OneHotEncoder()
onehotencoder = onehotencoder.fit(train_y)
onehot_encoded_train_y = onehotencoder.transform(train_y).toarray()
onehot_encoded_test_y = onehotencoder.transform(test_y).toarray()

exploration_results = []


n_features = train_x.shape[1]
categories = len(onehot_encoded_train_y[0])

test_acc = 0.0
model = Sequential()
interation = 0
while (
    test_acc < DNN_EXPLORATION_TARGET_VAL_ACCURACY
    and interation < DNN_EXPLORATION_MAX_ITER
):
    model = Sequential()
    model.add(InputLayer(input_shape=(n_features,)))
    model.add(BatchNormalization())
    dense_count = random.randint(
        DNN_EXPLORATION_HIDDEN_LAYERS_MIN, DNN_EXPLORATION_HIDDEN_LAYERS_MAX
    )
    dense_neurons = []
    for i in range(0, dense_count):
        neurons = random.randint(
            DNN_EXPLORATION_NEURONS_MIN, DNN_EXPLORATION_NEURONS_MAX
        )
        dense_neurons.append(neurons)
        model.add(Dense(neurons, activation="tanh"))
        model.add(BatchNormalization())
    model.add(Dense(categories, activation="sigmoid"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    model.fit(
        train_x,
        onehot_encoded_train_y,
        epochs=DNN_EPOCHS,
        batch_size=DNN_BATCH_SIZE,
        verbose=1,
        validation_data=(test_x, onehot_encoded_test_y),
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                patience=DNN_EARLY_STOPPING_PATIENCE,
                restore_best_weights=True,
            )
        ],
    )

    train_loss, train_acc = model.evaluate(train_x, onehot_encoded_train_y)
    test_loss, test_acc = model.evaluate(test_x, onehot_encoded_test_y)

    exploration_results.append(
        {
            "dense_count": dense_count,
            "dense_neurons": dense_neurons,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
        }
    )
    interation += 1

modelName = "dnn." + str(round(time.time()))
model.save(PATH_MODEL + modelName)

with open(PATH_EXPLORATION_DATA + modelName + ".exploration_results.json", "w") as f:
    json.dump(exploration_results, f, indent=4)

## 6. Modelanalyse des Learners [Kevin]

- Vorherige Auswahl eines Learners
- Feature Importance
- Korrelationsmatrix
- Konfusionsmatrix
- Post-Validation des Models mit auswählbaren Daten


In [ ]:
# SELECT LEARNER AND TEST DATA
featureDataDir = list(filter(lambda x: os.path.isfile(os.path.join(PATH_FEATURE_DATA, x)) and DEFAULT_RAW_DATA_TYPE in x.upper(), os.listdir(PATH_FEATURE_DATA)))
modelDir = os.listdir(PATH_MODEL)

modelDropdown = widgets.Dropdown(description="model")
modelDropdown.options = modelDir
selectedModelFile = None
def onTrainigFileChange(change):
    global selectedModelFile
    selectedModelFile = change['new']
modelDropdown.observe(onTrainigFileChange, names='value')
display(modelDropdown)

testFileDropdown = widgets.Dropdown(description="test data")
testFileDropdown.options = featureDataDir
selectedTestFile = None
def onTestFileChange(change):
    global selectedTestFile
    selectedTestFile = change['new']
testFileDropdown.observe(onTestFileChange, names='value')
display(testFileDropdown)


In [ ]:
# ANALYZE MODEL
from matplotlib import pyplot
from tensorflow import math as tfmath
import tensorflow_probability as tfp
import eli5
from eli5.sklearn import PermutationImportance

conf_matrix = tfmath.confusion_matrix(np.argmax(y_test, axis=1), np.argmax(result, axis=1))
pyplot.matshow(conf_matrix, 1)
for (x, y), value in np.ndenumerate(conf_matrix):
    pyplot.text(y, x, f"{value:.2f}", va="center", ha="center")
pyplot.show()


print('Test Accuracy: %.3f' % test_acc)


corr_matrix = tfp.stats.correlation(X_test)
pyplot.matshow(corr_matrix)
pyplot.show()

perm = PermutationImportance(model, scoring="neg_mean_squared_error", random_state=1).fit(X_test, y_test)
print(eli5.format_as_text(eli5.explain_weights(perm, feature_names=feature_names)))

## 7. Statische Interpretation des Resultats

- Welches Ergebnis haben wir erzielt und wie kann man es anwenden?
